In [1]:
%pip install beautifulsoup4 requests pandas langchain --user

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path
from urllib.parse import urlparse

import requests
from bs4 import BeautifulSoup

root_url = "https://airflow.apache.org/docs/apache-airflow/stable/"
root_response = requests.get(root_url)
root_html = root_response.content.decode("utf-8")
soup = BeautifulSoup(root_html, "html.parser")

root_url_parts = urlparse(root_url)
root_links = soup.find_all("a", attrs={"class": "reference internal"})

result = set()
for root_link in root_links:
    path = root_url_parts.path + root_link.get("href")
    path = str(Path(path).resolve())
    path = urlparse(path).path
    url = f"{root_url_parts.scheme}://{root_url_parts.netloc}{path}"
    result.add(url)
urls = list(result)
print(*urls, sep="\n")

https://airflow.apache.org/docs/apache-airflow/stable/migrations-ref.html
https://airflow.apache.org/docs/apache-airflow/stable/project.html
https://airflow.apache.org/docs/apache-airflow/stable/core-concepts/index.html
https://airflow.apache.org/docs/apache-airflow/stable/tutorial/index.html
https://airflow.apache.org/docs/apache-airflow/stable/ui.html
https://airflow.apache.org/docs/apache-airflow/stable/extra-packages-ref.html
https://airflow.apache.org/docs/apache-airflow/stable/license.html
https://airflow.apache.org/docs/apache-airflow/stable/best-practices.html
https://airflow.apache.org/docs/apache-airflow/stable/configurations-ref.html
https://airflow.apache.org/docs/apache-airflow/stable/authoring-and-scheduling/index.html
https://airflow.apache.org/docs/apache-airflow/stable/stable-rest-api-ref.html
https://airflow.apache.org/docs/apache-airflow/stable/security/index.html
https://airflow.apache.org/docs/apache-airflow/stable/howto/index.html
https://airflow.apache.org/docs/a

In [7]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://airflow.apache.org/docs/apache-airflow/stable/")
documents = loader.load()
documents[0]
# Conclusion: This loads a lot of unnecesary text and is poorly formated.

Document(page_content='\n\n\n  \n\n\n\n\nWhat is Airflow™? — Airflow Documentation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                            Community\n                        \n\n                            Meetups\n                        \n\n                            Documentation\n                        \n\n                            Use-cases\n                        \n\n                            Announcements\n                        \n\n                            Blog\n                        \n\n                            Ecosystem\n                        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                Community\n                            \n\n                                Meetups\n                            \n\n                                Documentation\n                            \n\n                                Use-cases\n    

In [26]:
response = requests.get("https://airflow.apache.org/docs/apache-airflow/stable/")
html = response.content.decode("utf-8")
soup = BeautifulSoup(root_html, "html.parser")

# Find all divs with class="section"
divs = soup.find_all("div", class_="section")

# Extract the text from each div that contains a 'pre' element
divs_with_pre = [div for div in divs if div.find("pre")]

# Example of a div:
example_div = divs_with_pre[0]

# Find all p tags within this div
p_tags = example_div.find_all("p")
p_texts = [p_tag.text.strip() for p_tag in p_tags]
all_p_text = " ".join(p_texts)

# Find all pre tags within this div
pre_tags = example_div.find_all("pre")
all_span_text = ""
code_snippet_start = "# python\n# code-snippet-start\n"
code_snippet_end = "\n# code-snippet-end"
for pre_tag in pre_tags:
    span_tags = pre_tag.find_all("span")
    span_texts = [span_tag.text for span_tag in span_tags]
    all_span_text = " ".join(span_texts)
    all_span_text = (
        "# python\n# code-snippet-start\n"
        f"{all_span_text.strip()}"
        "\n# code-snippet-end"
    )

print("Text:", all_p_text)
print(f"Char Length: {len(all_p_text)}")
print("", "=" * 100, "", sep="\n")
print("Code:", all_span_text)
print(f"Char Length: {len(all_span_text)}")

Text: The main characteristic of Airflow workflows is that all workflows are defined in Python code. “Workflows as
code” serves several purposes: Dynamic: Airflow pipelines are configured as Python code, allowing for dynamic pipeline generation. Extensible: The Airflow™ framework contains operators to connect with numerous technologies. All Airflow components are extensible to easily adjust to your environment. Flexible: Workflow parameterization is built-in leveraging the Jinja templating engine. Take a look at the following snippet of code: Here you see: A DAG named “demo”, starting on Jan 1st 2022 and running once a day. A DAG is Airflow’s representation of a workflow. Two tasks, a BashOperator running a Bash script and a Python function defined using the @task decorator >> between the tasks defines a dependency and controls in which order the tasks will be executed Airflow evaluates this script and executes the tasks at the set interval and in the defined order. The status
of the “

In [30]:
def _parse_url(url):
    response = requests.get(url)
    html = response.content.decode("utf-8")
    soup = BeautifulSoup(root_html, "html.parser")
    return soup


def extract_sections(url, exclude_no_code=True):
    soup = _parse_url(url)
    # Find all divs with class="section"
    divs = soup.find_all("div", class_="section")
    if exclude_no_code:
        # Extract the text from each div that contains a 'pre' element
        code_divs = [div for div in divs if div.find("pre")]
        return code_divs
    return divs


def _extract_text_from_div(div):
    # Find all p tags within this div
    p_tags = div.find_all("p")
    p_texts = [p_tag.text.strip() for p_tag in p_tags]
    all_p_text = " ".join(p_texts)
    return all_p_text


def _extract_code_from_div(div):
    # Find all pre tags within this div
    pre_tags = div.find_all("pre")
    all_span_text = ""
    code_snippet_start = "# python\n# code-snippet-start\n"
    code_snippet_end = "\n# code-snippet-end"
    for pre_tag in pre_tags:
        span_tags = pre_tag.find_all("span")
        span_texts = [span_tag.text for span_tag in span_tags]
        all_span_text = " ".join(span_texts)
        all_span_text = (
            "# python\n# code-snippet-start\n"
            f"{all_span_text.strip()}"
            "\n# code-snippet-end"
        )
    return all_span_text

In [33]:
dataset = []

for url in urls:
    for section in extract_sections(url):
        dataset.append(
            {
                "url": url,
                "text": _extract_text_from_div(section),
                "code": _extract_code_from_div(section),
            }
        )

print("Results:")
print(*dataset, sep="\n\n")

Results:
{'url': 'https://airflow.apache.org/docs/apache-airflow/stable/migrations-ref.html', 'text': 'The main characteristic of Airflow workflows is that all workflows are defined in Python code. “Workflows as\ncode” serves several purposes: Dynamic: Airflow pipelines are configured as Python code, allowing for dynamic pipeline generation. Extensible: The Airflow™ framework contains operators to connect with numerous technologies. All Airflow components are extensible to easily adjust to your environment. Flexible: Workflow parameterization is built-in leveraging the Jinja templating engine. Take a look at the following snippet of code: Here you see: A DAG named “demo”, starting on Jan 1st 2022 and running once a day. A DAG is Airflow’s representation of a workflow. Two tasks, a BashOperator running a Bash script and a Python function defined using the @task decorator >> between the tasks defines a dependency and controls in which order the tasks will be executed Airflow evaluates th